# Motivation
论文原文：[《Q-learning》](https://link.springer.com/article/10.1007/bf00992698)

参考学习地址：
* https://d2l.ai/chapter_reinforcement-learning/qlearning.html
* [强化学习入门笔记——Q​-learning从理论到实践](https://zhuanlan.zhihu.com/p/365814943)
* [如何用简单例子讲解 Q - learning 的具体过程？](https://www.zhihu.com/question/26408259/answer/123230350)

代码库：
* https://enhuiz.github.io/flappybird-ql/
* https://github.com/ChengpengLi1003/Q-learning/blob/main/Q-learning.py

强化学习中有状态（state），动作（action），奖励（reward）三要素，智能体（agent）根据当前状态来采取行动，记录被反馈的奖励，以便下次再到相同状态时能采取更优的动作。按不同标准把强化学习算法分为不同类别：
* 根据RL的agent是否显示学习一个策略进行分类
  * 基于价值的方法（value-based）：agent显式学习价值函数，隐式学习策略（也即策略从学习的价值函数中推算获取）
  * 基于策略的方法（policy-based）：agent直接学习policy
  * 结合value-based和policy-based方法：演员-评论家（Actor-Critic）方法，需要显式地学习价值函数和策略
* 通过agent与环境交互采集数据的策略与用于学习更新策略是否一致进行分类
  * 同轨策略（on-policy）：使用策略$\pi$学习，同时使用策略$\pi$与环境交互采集数据
  * 离轨策略（off-policy）：有一种行为策略$\mu$与环境进行交互采集数据，而使用另一种目标策略$\pi$学习最优策略
* 按是否学习环境的函数（一般指状态转移函数）分为model-based（基于模型）和model-free（无模型）
* 按agent是否和环境交互分为online和offline

<img style="display: block; margin: 0 auto;" src="../../../assets/images/q_learning.png" width = "600" height = "300" align=center />

Q-learning是一种value-based，off-policy，model-free，offline模型。

## MDP
智能体和环境交互往往通过马尔科夫决策过程（Markov Decision Process）建模。由五元组$(S,A,P,r,\gamma)$表示：
* $S$为状态空间
* $A$为动作空间
* $P$为状态转移函数，用$P: S\times A \rightarrow \Delta (S)$表示，其中$\Delta (S)$表示在$S$上的概率分布。例如在状态空间和动作空间有限时，$P(s'|s,a)$表示在$s$状态下采取动作$a$后状态转移到$s'$的概率
* $R$表示奖励函数，用$R: S\times A \rightarrow \Delta(R)$，其中$\Delta(R)$表示在$R$上的概率分布。通常奖励信号由人为设计的确定函数。$R(s,a)$表示在状态$s$下采取动作$a$后获得的奖励
* 以上四个部分构成了一个MDP：$(S,A,P,r)$，为了个行进步数增加一个惩罚，通常会引入折扣因子$\gamma <1$，此时收益率为$R(t)=\sum_{t=0}^{\infty}\gamma^tr_t$

在一个MDP过程中，智能体的目标是学习到一个策略（Policy），用以指导在每一个状态$s_t$下采取动作$a_t$。记作$\pi:S\rightarrow \Delta(A)$，其中$\Delta(A)$代表在动作空间$A$上的概率分布，可以把在状态$s$下执行策略后采取某个动作$a$的概率表示为$P(a_t=a|s_t=s)$记作$\pi(a|s)$。

强化学习任务通常分为两类：
* 分幕式任务：存在终止时刻，比如围棋，最后有胜负达到终止态。把从开始到终止的过程称为一幕
* 持续性任务：智能体和环境可以持续不断地发生交互的任务，比如机器人控制

两类任务的回报统一定义为$G_t=\sum_{k=t}^{\infty}\gamma^{k-t}r_t$，在回报的基础上，可以结合策略来定义策略的“状态价值函数”和“动作价值函数”，来衡量执行某个策略时，某个状态能获得回报的期望和某个状态执行某个动作后能获得回报的期望。
* 策略的价值函数：$V_{\pi}:S\rightarrow \mathbb{R}$，即在当前状态是$\forall s\in S$，执行策略$\pi$获得的回报期望 $V_{\pi}(s)=\mathbb{E}_{s_t,a_t}[\sum_{t=0}^{\infty}\gamma^tR(s_t,a_t)|s_0=s]$
* 策略的动作价值函数：$Q_{\pi}:S\times A\rightarrow \mathbb{R}$，即在当前状态是$\forall s\in S$采取动作$a\in A$后，在执行策略$\pi$能获得回报的期望
$$
\begin{align}
Q_{\pi}(s,a)&=R(s,a)+\mathbb{E}_{s_t,a_t}[\sum_{t=0}^{\infty}\gamma^tR(s_t,a_t)|s_0=s,a_0=a]\\
&=\mathbb{E}_{s_1}[R(s,a) + \gamma V_{\pi}(s_1)|s_0=s,a_0=a]
\end{align}
$$
* 最优策略$\pi^*$是指能使$V_{\pi^*}(s)\ge V_{\pi}(s), \forall s\in S$。最优策略下，分别记状态价值函数和动作价值函数为$V^*(s)=\underset{\pi}{max}V_{\pi}(s)$和$Q^*(s,a)=\mathbb{E}_{s_1}[R(s,a) + \gamma V^*(s_1)|s_0=s,a_0=a]$
* 贝尔曼方程：对于策略$\pi$，其状态价值函数和动作价值函数可相互表示，且因此可获得迭代形式的贝尔曼方程
$$
\begin{align}
V_{\pi}(s) = \sum_{a\in A}\pi (a|s)Q_{\pi}(s,a)\\
Q_{\pi} (s,a) = R(s,a) + \gamma \sum_{s'\in S}P(s'|s,a)V_{\pi}(s')
\end{align}
$$
* 贝尔曼最优方程：将最优策略下的价值函数和动作价值函数关系记作贝尔曼最优方程
$$
\begin{align}
V^*(s) = \underset{a\in A}{max} Q^*(s,a)\\
Q^* (s,a) = R(s,a) + \gamma \sum_{s'\in S}P(s'|s,a)V^*(s')
\end{align}
$$
* 贝尔曼最优算子：$\Gamma:\R^{S\times A}\rightarrow \R^{S\times A}$使得$(\Gamma f)(s,a):=R(s,a) + \gamma \sum_{s'\in S}P(s'|s,a)\underset{a\in A}{max}f(s',a)$，注意到$Q^*$是贝尔曼最优算子的不动点
$$
\begin{align}
\Gamma Q^*(s, a) &= R(s,a) + \gamma \sum_{s'\in S}P(s'|s,a)\underset{a\in A}{max}Q^*(s',a) \\
&= R(s,a) + \gamma \sum_{s'\in S}P(s'|s,a)V^*(s') \\
&= Q^*(s,a)
\end{align}
$$

三个定理
1. 策略提升定理：对任两个策略$\pi$和$\pi '$，$(\forall s\in S, \underset{a\tilde \pi ' (s)}{\mathbb{E}}[Q_{\pi}(s,a)] \ge \underset{a\tilde \pi (s)}{\mathbb{E}}[Q_{\pi}(s,a)]) \Rightarrow (\forall s\in S, V_{\pi '}(s) \ge V_{\pi}(s))$。如果至少存在一个状态使得上式左边的不等式严格成立，那么也至少存在一个状态使得上式右边的不等式严格成立
2. 贝尔曼最优条件：一个策略$\pi$是最优策略，当且仅当对任意状态动作对$(s,a)\in S\times A$满足$a\in argmax_{a'\in A}Q_{\pi}(s,a')$
3. 最优策略存在定理：任意一个有限MDP中，都存在最优的确定性策略

## Q表
Q为动作效用函数（action-utility function），用于评价在特定状态下采取这个动作的优劣，是智能体的记忆。理想状态下，训练完成后会有一张完美的Q表格，记录所有可枚举状态及其对应的效用值。根据上最优策略的介绍，最优策略$\pi ^*$可通过$Q^*$获取（$\pi^*(s)=argmax_{a\in A}Q^*(s,a), s\in S$）。那么现在的问题是如何学习一个$Q_\pi$函数，使其收敛到$Q^*$。在有限MDP中，$S$和$A$是有限集合，$Q_\pi(s,a)$可以看作存储着$Q$值的$|S|$行$|A|$列的表格，也即$Q$表。

### $\epsilon$贪心策略
更新$Q$表，希望agent既可以使用和环境交互采集数据获得的经验，又可以探索一些未知的状态动作。贪心策略指agent每次采取动作受$\pi(s)=argmax_{a\in A}Q_t(s,a)$控制，其中$Q_t(s,a)$是在时刻$t$，agent拥有的$Q$表中$(s,a)$对应的$Q$值。由于每次选择动作都使得在现有经验下，后续的收益达到最大，所以称为贪心策略。

而$\epsilon$贪心是指，智能体每次以$\epsilon$概率执行随机动作，以$(1-\epsilon)$的概率执行贪心策略。$\epsilon$也即探索系数。

### 增量式实现
增量式更新方法一般形式为：$新估计值 \leftarrow 旧估计值 + 步长 \times [目标 - 旧估计值]$，也即向着估计误差减小的方向手链。在Q-learning中增量更新为：
$$
\begin{align}
Q(s,a) \leftarrow (1- \alpha)Q(s,a) + \alpha (R(s,a) + \gamma \underset{\alpha}{max} Q(S',a))
\end{align}
$$
其中，$\alpha$是步长，$S'$是当前状态执行行为策略后观测到了下一个状态。状态更新时会同时考虑眼前利益$R$和记忆中的利益$\underset{\alpha}{max} Q(S',a)$，$\gamma$越大，agent更注重以往的经验，反之更注重眼前利益。迭代的贝尔曼方程为
$$
\begin{align}
Q_\pi(s,a) = R(s,a) + \gamma \sum_{s'\in S}P(s'|s,a)\sum_{a'\in A}\pi(a'|s')Q_\pi(s',a')
\end{align}
$$
